### Experiment: Normalization

**Question**: What is the performance on one dataset for each of the models?

**Hypothesis**: 

**Result**:

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import xarray as xr
from hmpai.utilities import print_results
from hmpai.pytorch.models import *
from hmpai.pytorch.training import k_fold_cross_validate
from hmpai.normalization import *
from hmpai.data import SAT1_STAGES_ACCURACY